In [1]:
import geopandas as gpd
import pandas as pd
from pytz import timezone
import folium
from folium.plugins import TimestampedGeoJson
from datetime import datetime

# VTEC dictionaries (already provided by you)
VTEC_PHENOMENA = {
    "AF": "Ashfall", "AS": "Air Stagnation", "BH": "Beach Hazard", "BS": "Blowing Snow",
    "BW": "Brisk Wind", "BZ": "Blizzard", "CF": "Coastal Flood", "CW": "Cold Weather",
    "DF": "Debris Flow", "DS": "Dust Storm", "DU": "Blowing Dust", "EC": "Extreme Cold",
    "EH": "Excessive Heat", "EW": "Extreme Wind", "FA": "Flood", "FF": "Flash Flood",
    "FG": "Dense Fog", "FL": "Flood", "FR": "Frost", "FW": "Red Flag", "FZ": "Freeze",
    "UP": "Freezing Spray", "GL": "Gale", "HF": "Hurricane Force Wind", "HI": "Inland Hurricane",
    "HS": "Heavy Snow", "HT": "Heat", "HU": "Hurricane", "HW": "High Wind", "HY": "Hydrologic",
    "HZ": "Hard Freeze", "IP": "Sleet", "IS": "Ice Storm", "LB": "Lake Effect Snow and Blowing Snow",
    "LE": "Lake Effect Snow", "LO": "Low Water", "LS": "Lakeshore Flood", "LW": "Lake Wind",
    "MA": "Marine", "MF": "Marine Dense Fog", "MH": "Marine Ashfall", "MS": "Marine Dense Smoke",
    "RB": "Small Craft for Rough", "RP": "Rip Currents", "SB": "Snow and Blowing", "SC": "Small Craft",
    "SE": "Hazardous Seas", "SI": "Small Craft for Winds", "SM": "Dense Smoke", "SN": "Snow",
    "SQ": "Snow Squall", "SR": "Storm", "SS": "Storm Surge", "SU": "High Surf", "SV": "Severe Thunderstorm",
    "SW": "Small Craft for Hazardous Seas", "TI": "Inland Tropical Storm", "TO": "Tornado", "TR": "Tropical Storm",
    "TS": "Tsunami", "TY": "Typhoon", "WC": "Wind Chill", "WI": "Wind", "WS": "Winter Storm",
    "WW": "Winter Weather", "ZF": "Freezing Fog", "ZR": "Freezing Rain"
}

VTEC_SIGNIFICANCE = {
    "W": "Warning", "Y": "Advisory", "A": "Watch", "S": "Statement", "O": "Outlook", "N": "Synopsis", "F": "Forecast"
}

NWS_COLORS = {
    "AF.W": "#A9A9A9", "AF.Y": "#696969", "AS.O": "#808080", "AS.Y": "#808080", "BH.S": "#40E0D0",
    "BW.Y": "#D8BFD8", "BZ.A": "#ADFF2F", "BZ.W": "#FF4500", "CF.A": "#66CDAA", "CF.S": "#6B8E23",
    "CF.W": "#228B22", "CF.Y": "#7CFC00", "DS.W": "#FFE4C4", "DS.Y": "#BDB76B", "DU.W": "#FFE4C4",
    "DU.Y": "#BDB76B", "EC.A": "#0000FF", "EC.W": "#0000FF", "EH.A": "#800000", "EH.W": "#C71585",
    "EH.Y": "#800000", "EW.W": "#FF8C00", "FA.A": "#2E8B57", "FA.W": "#00FF00", "FA.Y": "#00FF7F",
    "FF.A": "#2E8B57", "FF.S": "#8B0000", "FF.W": "#8B0000", "FG.Y": "#708090", "FL.A": "#2E8B57",
    "FL.S": "#00FF00", "FL.W": "#00FF00", "FL.Y": "#00FF7F", "FR.Y": "#6495ED", "FW.A": "#FFDEAD",
    "FW.W": "#FF1493", "FZ.A": "#00FFFF", "FZ.W": "#483D8B", "GL.A": "#FFC0CB", "GL.W": "#DDA0DD",
    "HF.A": "#9932CC", "HF.W": "#CD5C5C", "HT.Y": "#FF7F50", "HU.A": "#FF00FF", "HU.S": "#FFE4B5",
    "HU.W": "#DC143C", "HW.A": "#B8860B", "HW.W": "#DAA520", "HY.Y": "#00FF7F", "HZ.A": "#4169E1",
    "HZ.W": "#9400D3", "IS.W": "#8B008B", "LE.A": "#87CEFA", "LE.W": "#008B8B", "LE.Y": "#48D1CC",
    "LO.Y": "#A52A2A", "LS.A": "#66CDAA", "LS.S": "#6B8E23", "LS.W": "#228B22", "LS.Y": "#7CFC00",
    "LW.Y": "#D2B48C", "MA.S": "#FFDAB9", "MA.W": "#DB7093", "MF.Y": "#708090", "MH.Y": "#696969",
    "MS.Y": "#F0E68C", "RB.Y": "#D8BFD8", "RP.S": "#40E0D0", "SC.Y": "#D8BFD8", "SE.A": "#483D8B",
    "SE.W": "#D8BFD8", "SI.Y": "#D8BFD8", "SM.Y": "#F0E68C", "SQ.W": "#C71585", "SR.A": "#FFE4B5",
    "SR.W": "#9400D3", "SS.A": "#DB7FF7", "SS.W": "#C0C0C0", "SU.W": "#228B22", "SU.Y": "#BA55D3",
    "SV.A": "#DB7093", "SV.W": "#FFA500", "SW.Y": "#D8BFD8", "TO.A": "#FFFF00", "TO.W": "#FF0000",
    "TR.A": "#F08080", "TR.S": "#FFE4B5", "TR.W": "#B22222", "TS.A": "#FF00FF", "TS.W": "#FD6347",
    "TS.Y": "#D2691E", "TY.A": "#FF00FF", "TY.W": "#DC143C", "UP.A": "#4682B4", "UP.W": "#8B008B",
    "UP.Y": "#8B008B", "WC.A": "#5F9EA0", "WC.W": "#B0C4DE", "WC.Y": "#AFEEEE", "WI.Y": "#D2B48C",
    "WS.A": "#4682B4", "WS.W": "#FF69B4", "WW.Y": "#7B68EE", "ZF.Y": "#008080", "ZR.Y": "#DA70D6"
}

# Paths to the shapefiles
nws_shapefile_path = 'nws_data/wwa_202406260000_202406271200.shp'
ny_boundary_shapefile_path = 'state/State.shp'

# Load the NWS data
nws_data = gpd.read_file(nws_shapefile_path)

# Load the New York State boundary shapefile
ny_boundary = gpd.read_file(ny_boundary_shapefile_path)

/tmp/ipykernel_1858345/1540582628.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Function to convert UTC to Eastern Time
def convert_to_eastern(utc_time):
    # Define timezones
    eastern = timezone('US/Eastern')
    utc = timezone('UTC')

    # Localize UTC time to make it timezone-aware
    utc_time = pd.to_datetime(utc_time).tz_localize('UTC')
    
    # Convert to Eastern Time
    eastern_time = utc_time.astimezone(eastern)
    return eastern_time

# Assuming nws_data is already loaded and has an 'ISSUED' column in UTC
nws_data['ISSUED_ET'] = nws_data['ISSUED'].apply(convert_to_eastern)

# Display the first few rows to check the conversion
#print(nws_data[['ISSUED', 'ISSUED_ET']].head())

In [3]:
# Assuming nws_data and ny_boundary are already loaded and reprojected to UTM Zone 18N
# Reproject to UTM zone 18N (EPSG:32618)
nws_data = nws_data.to_crs(epsg=32618)
ny_boundary = ny_boundary.to_crs(epsg=32618)

# Spatial join to find the intersection
intersection = gpd.overlay(nws_data, ny_boundary, how='intersection')

# Filter by minimum area within NY (adjust threshold as needed)
min_area_threshold = 10e6  # 1,000,000 square meters (1 square kilometer)
intersection['area'] = intersection.geometry.area
filtered_intersection = intersection[intersection['area'] >= min_area_threshold]

# Display information about the filtered data
print(f"Number of polygons before filtering: {intersection.shape[0]}")
print(f"Number of polygons after filtering: {filtered_intersection.shape[0]}")

Number of polygons before filtering: 79
Number of polygons after filtering: 67


In [4]:
# Filter out only the 'NEW' status entries
new_warnings = filtered_intersection[filtered_intersection['STATUS'] == 'NEW']

# Display information about the filtered data
print(f"Number of polygons with 'NEW' status: {new_warnings.shape[0]}")

Number of polygons with 'NEW' status: 23


In [5]:
# Decode PHENOM column using VTEC_PHENOMENA dictionary
new_warnings['PHENOM_DESCRIPTION'] = new_warnings['PHENOM'].map(VTEC_PHENOMENA)

# Decode SIG column using VTEC_SIGNIFICANCE dictionary
new_warnings['SIG_DESCRIPTION'] = new_warnings['SIG'].map(VTEC_SIGNIFICANCE)

# Display the decoded columns
print(new_warnings[['PHENOM', 'PHENOM_DESCRIPTION', 'SIG', 'SIG_DESCRIPTION']].head())

   PHENOM   PHENOM_DESCRIPTION SIG SIG_DESCRIPTION
6      MA               Marine   W         Warning
7      MA               Marine   W         Warning
8      SV  Severe Thunderstorm   W         Warning
9      SV  Severe Thunderstorm   W         Warning
10     SV  Severe Thunderstorm   W         Warning


/knight/mamba_aug23/envs/aug23_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/knight/mamba_aug23/envs/aug23_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
# Group by ISSUED_ET, PHENOM_DESCRIPTION, SIG_DESCRIPTION, and keep the row with the maximum AREA_KM2 for each group
max_area_warnings = new_warnings.loc[
    new_warnings.groupby(['ISSUED_ET', 'PHENOM_DESCRIPTION', 'SIG_DESCRIPTION'])['AREA_KM2'].idxmax()
]

# Display the cleaned and filtered warnings
#print(max_area_warnings.head())

In [7]:
summary = max_area_warnings.groupby(['PHENOM_DESCRIPTION', 'SIG_DESCRIPTION']).size().reset_index(name='Count')

In [8]:
# Centered around New York State
ny_center = (42.9359825, -75.6101884)  # Adjust coordinates as needed
map_ny = folium.Map(location=ny_center, zoom_start=7)

# Filter warnings issued on June 26th
warnings_june_26 = max_area_warnings[max_area_warnings['ISSUED_ET'].dt.date == datetime(2024, 6, 26).date()]

# Convert ISSUED_ET to string
warnings_june_26['ISSUED_ET'] = warnings_june_26['ISSUED_ET'].astype(str)

# Convert GeoDataFrame to GeoJSON
warnings_geojson = warnings_june_26.to_crs(epsg='4326').to_json()

In [9]:
# Function to style each feature (polygon)
def style_function(feature):
    phenom = feature['properties']['PHENOM']
    sig = feature['properties']['SIG']
    color = NWS_COLORS.get(f"{phenom}.{sig}", "#FF0000")  # Default to red if color not found
    return {
        'fillOpacity': 0.3,
        'weight': 1,
        'color': color,
        'fillColor': color,
    }

In [10]:
# Add GeoJSON layer to map
folium.GeoJson(
    warnings_geojson,
    name='Warnings on June 26th',
    style_function=style_function
).add_to(map_ny)

In [11]:
map_ny